formation energy per atom 의 결과물을 reproduce 하는 코드

MAE 와 R2 score 분석을 통해 모델 성능 확인

In [ ]:
# 할당받은 GPU 성능 확인
!nvidia-smi

Wed Dec 29 04:22:23 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install pymatgen==2020.11.11

In [ ]:
import pymatgen
dir(pymatgen)

# CGCNN 모델 학습

In [ ]:
import time
import sys
import os
import torch
import sklearn
import pandas as pd
import shutil

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn import metrics

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

In [ ]:
def check_time(init_time, fin_time):
    time_interval = fin_time - init_time
    hour = time_interval // 3600
    time_interval = time_interval - 3600 * hour
    min = time_interval // 60
    time_interval = time_interval - 60 * min
    sec = round(time_interval)

    return("{} hours {} mins {} secs".format(hour, min, sec))

In [ ]:
def data_plot(path = "/content/drive/MyDrive/packages/cgcnn/cgcnn-master/results/formation_energy/test_results.csv"):
    dt = pd.read_csv(path, header = None)
    predicted_data = list(dt.iloc[:,2])
    real_data = list(dt.iloc[:,1])

    test_number = len(predicted_data)

    RMSE = mean_squared_error(real_data, predicted_data, squared = False)
    MAE = mean_absolute_error(real_data, predicted_data)
    r2 = r2_score(real_data, predicted_data)
    
    fig, ax = plt.subplots()

    ax.hist2d(real_data, predicted_data, norm=LogNorm(), bins=(100,100), cmap='Blues', alpha=0.7)

    ax.plot(ax.get_xlim(), ax.get_xlim(), 'k--')

    ax.set_xlabel('MP band gap (eV)')
    ax.set_ylabel('ML band gap (eV)')

    fig.set_size_inches(5, 5)
    fig.tight_layout()

    plt.show()

    print("\n"+"test number : {}, RMSE : {}, MAE : {}, R2 : {}".format(test_number, RMSE, MAE, r2))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
my_path = '/content/drive/MyDrive/packages/cgcnn/cgcnn-master/data/formation_energy_training/formation_energy'
sys.path.insert(0, my_path)

In [ ]:
os.chdir('/content/drive/MyDrive/packages/cgcnn/cgcnn-master/data/formation_energy_training/formation_energy')

In [ ]:
dir_list = os.listdir("/content/drive/MyDrive/packages/cgcnn/cgcnn-master/data/formation_energy_training/formation_energy")

In [ ]:
len(dir_list)

3

# 학습 시작

In [ ]:
time_1 = time.time()

!python ../../../main.py ./ --train-ratio 0.6 --val-ratio 0.2 --test-ratio 0.2 --epochs 1000 --n-conv 4 --h-fea-len 32 --lr 0.02 --lr-milestones 800 --print-freq 120

time_2 = time.time()

cif_json directory : ./cif_group.json
Epoch: [0][0/110]	Time 11.699 (11.699)	Data 11.613 (11.613)	Loss 1.1023 (1.1023)	MAE 0.900 (0.900)
Test: [0/37]	Time 12.136 (12.136)	Loss 0.0493 (0.0493)	MAE 0.164 (0.164)
 * MAE 0.172
Epoch: [1][0/110]	Time 0.116 (0.116)	Data 0.042 (0.042)	Loss 0.0661 (0.0661)	MAE 0.183 (0.183)
Test: [0/37]	Time 0.040 (0.040)	Loss 0.0435 (0.0435)	MAE 0.151 (0.151)
 * MAE 0.163
Epoch: [2][0/110]	Time 0.104 (0.104)	Data 0.036 (0.036)	Loss 0.0401 (0.0401)	MAE 0.154 (0.154)
Test: [0/37]	Time 0.042 (0.042)	Loss 0.4364 (0.4364)	MAE 0.213 (0.213)
 * MAE 0.175
Epoch: [3][0/110]	Time 0.348 (0.348)	Data 0.277 (0.277)	Loss 0.0602 (0.0602)	MAE 0.216 (0.216)
Test: [0/37]	Time 0.042 (0.042)	Loss 0.0308 (0.0308)	MAE 0.144 (0.144)
 * MAE 0.159
Epoch: [4][0/110]	Time 0.110 (0.110)	Data 0.037 (0.037)	Loss 0.0725 (0.0725)	MAE 0.231 (0.231)
Test: [0/37]	Time 0.043 (0.043)	Loss 0.0329 (0.0329)	MAE 0.126 (0.126)
 * MAE 0.125
Epoch: [5][0/110]	Time 0.109 (0.109)	Data 0.041 (0.041)	Loss 

In [ ]:
check_time(time_1, time_2)

'3.0 hours 46.0 mins 47 secs'

In [ ]:
data_plot("/content/drive/MyDrive/packages/cgcnn/cgcnn-master/results/formation_energy/test_results.csv")

비교적 최근에 추가된 mp-id의 구조들 931개로 테스트